In [2]:
import sys
import importlib
sys.path.append(r'../../pykneer')
sys.path.append(r'../pykneer')
import pykneer_io as io
importlib.reload(io)
import preprocessing_for_nb as prep
importlib.reload(prep)
import segmentation_sa_for_nb as segm
importlib.reload(segm)

import SimpleITK as sitk
import matplotlib.pyplot as plt

from preparing_images import prepare_image_and_list

In [3]:
# process list
# first in TP as reference
caselist = [
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
]

prepare_image_and_list(caselist)

ahaknee0tp29  exist
dcm path for TPid 0 Z:/OAI Baseline/ge0/9189303/10397606
Loading QVJ Y:/0tp29\cascade/0/9189303/10397606L\E10397606L_L.QVJ
Adding QVS E10397606LS101_L.QVS
coronal Z:/OAI Baseline/ge0/9189303/10397606/001.dcm
no such contour OuterWall
No lumen contour for frappe dcm slice 75
ahaknee12tp29  exist
dcm path for TPid 1 Z:/OAI 12 Months/2.C.2/9189303/20060522/11151205
Loading QVJ Y:/12tp29\cascade/12/9189303/20060522L\E20060522L_L.QVJ
Adding QVS E20060522LS101_L.QVS
coronal Z:/OAI 12 Months/2.C.2/9189303/20060522/11151205/001.dcm
no such contour OuterWall
No lumen contour for frappe dcm slice 75
ahaknee18tp29  exist
cannot find dcm path for TPid 2
ahaknee24tp29  exist
cannot find dcm path for TPid 3
ahaknee30tp29  exist
cannot find dcm path for TPid 4
ahaknee36tp29  exist
cannot find dcm path for TPid 5
ahaknee48tp29  exist
cannot find dcm path for TPid 6
ahaknee72tp29  exist
cannot find dcm path for TPid 8
ahaknee96tp29  exist
cannot find dcm path for TPid 10
ahaknee0tp2

In [4]:
input_file_name           = ".\image_list_preprocessing.txt"
n_of_cores                = 1 # change the number of cores according to your computer
intensity_standardization = 0

# Image information
image_data = io.load_image_data_preprocessing(input_file_name)

ref = sitk.ReadImage('../../reference/newsubject/reference.mha')

# Spatial standardization
# Step 0: Read the dicom images
prep.read_dicom_stack(image_data, n_of_cores)
# Save image header
prep.print_dicom_header(image_data, n_of_cores)    # save to *.txt
    
# Step 1: Change orientation to RAI
#prep.axial_to_sagittal(image_data, n_of_cores)
prep.orientation_to_rai(image_data, n_of_cores)

# Step 2: Change laterality if knee is right
prep.flip_rl(image_data, n_of_cores)
# Step 3: Set image origin to (0,0,0)
prep.origin_to_zero(image_data, n_of_cores)    # Images are saved as *_orig.mha and they are anonymized

# 4. Correct magnetic field inhomogeneities
# Magnetic fields inhomogeneities create grey shades on images. This correction removes these shades. 
# This is the longest step of the processing. It can take up to 15-20 min on a standard PC or laptop
if intensity_standardization == 1:
    prep.field_correction(image_data, n_of_cores)
    prep.rescale_to_range(image_data, n_of_cores)  # 5. Rescale intensities to [0 100]
    prep.edge_preserving_smoothing(image_data, n_of_cores)  # 6. Edge preserving smoothing

9189303L_TP0
9189303L_TP1
9941446R_TP0
9941446R_TP1
9941446R_TP3
9941446R_TP5
9941446R_TP6
9941446R_TP8
9941446R_TP10
-> information loaded for 9 subjects
-> Dicom images read
-> The total time was 9.52 seconds (about 0 min)
-> Dicom headers written
-> The total time was 3.07 seconds (about 0 min)
-> Image orientation changed
-> The total time was 4.25 seconds (about 0 min)
-> Image laterality changed for right images
-> The total time was 2.11 seconds (about 0 min)
-> Image origin changed
-> _orig.mha images saved
-> The total time was 2.94 seconds (about 0 min)


In [5]:
input_file_name = "./image_list_newsubject.txt"
modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)
segm.prepare_reference(image_data)
### NOTE!!! reference_fv is dummy!!!

# Segment Bone
# 1. Register image to reference
segm.register_bone_to_reference(image_data, n_of_cores)
# 2. Invert transformations
segm.invert_bone_transformations(image_data, n_of_cores)
# 3. Warp reference mask to moving image
segm.warp_bone_mask(image_data, n_of_cores)

# Segment cartilage
segm.register_cartilage_to_reference(image_data, n_of_cores)
segm.invert_cartilage_transformations(image_data, n_of_cores)
segm.warp_cartilage_mask(image_data, n_of_cores)

-> image information loaded
reference.mha
-> Reference preparation completed
-> Registration completed
-> The total time was 1.08 seconds (about 0 min)
-> Inversion completed
-> The total time was 1.04 seconds (about 0 min)
-> Warping completed
-> The total time was 1.03 seconds (about 0 min)
-> Registration completed
-> The total time was 1.05 seconds (about 0 min)
-> Inversion completed
-> The total time was 1.02 seconds (about 0 min)
-> Warping completed
-> The total time was 1.06 seconds (about 0 min)


In [ ]:
input_file_name = "./image_list_longitudinal.txt"
modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)
segm.prepare_reference(image_data)

segm.register_bone_to_reference(image_data, n_of_cores)
# segm.invert_bone_transformations(image_data, n_of_cores)
# segm.warp_bone_mask(image_data, n_of_cores)

# segm.register_vessel_to_reference(image_data, n_of_cores)
# segm.invert_vessel_transformations(image_data, n_of_cores)
# segm.warp_vessel_mask(image_data, n_of_cores)